本程序封装于函数ObjDet_Tom()中。
程序的执行环境和依赖的库为：Windows 10、Python 3.7.0、OpenCV-Python 3.4.2.17。

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Jul 28 19:51:13 2019

@author: Tom
"""

import numpy as np
import cv2 as cv

def ObjDet_Tom(filename1, filename2):

以下为运行本函数所需预先设置的参数，具体意义详见下文

In [ ]:
    STEP = (5, 5)
    COUNT_OF_BEST_MATCHES_PER_DESC = 2
    LOWE_RATIO = 0.5
    MIN_GOOD_MATCH_RATIO = 0.1
    DISTANCE_COEFFICIENT = 0.5
    SEARCH_COEFFICIENT = 0.6
    

读取大图（原图像）和小图（目标图像）

In [ ]:
    original = cv.imread(filename1)
    cv.imshow('Original', original)
    print("Shape of original image: {}".format(original.shape))
    
    target = cv.imread(filename2)
    cv.imshow('Target', target)
    print("Shape of target image: {}".format(target.shape))
    

将原图像和目标图像转换为灰度图像

In [ ]:
    originalGray = cv.cvtColor(original, cv.COLOR_BGR2GRAY)
    
    targetGray = cv.cvtColor(target, cv.COLOR_BGR2GRAY)
    

获取原图像和目标图像的宽度和高度。若目标图像的宽度和高度超过原图像的，则告警并立即返回。

In [ ]:
    h_o, w_o = original.shape[:2]
    h_t, w_t = target.shape[:2]
    
    if w_t > w_o:
        print("Width of the small image ({}) is larger than that of the large image ({})!".format(w_t, w_o))
        return []
    
    if h_t > h_o:
        print("Height of the small image ({}) is larger than that of the large image ({})!".format(h_t, h_o))
        return []
    

针对目标图像，使用SIFT特征检测器检测特征点和计算描述子。因为题目提供的图像的定位目标的尺寸很小，且存在其它颜色的同形状飞行器，为增加检测到的特征点数目，并避免匹配至其它颜色的飞行器，此处按图像的BGR三通道分别进行检测和计算。

**SIFT特征检测的优点是在空间和尺度上定位精确，因为其基于浮点内核计算特征点；但其计算量较大，花费的时间较多。此处也可使用SURF特征检测。SURF特征检测的优点是检测速度很快，适合实时视频流等场景；但定位没有SIFT算法精确。此外，也可以采用Harris、FAST、ORB等算法。**

In [ ]:
    sift = cv.xfeatures2d.SIFT_create()
    
    cntkeypointsT = 0
    keypointsTList = []
    descTList = []
    
    for c in range(3):
        keypoints, desc = sift.detectAndCompute(target[..., c], None)
        print("Count of keypoints[{:d}]: {:d}".format(c, len(keypoints)))
        print("Shape of descriptors[{:d}]: {}".format(c, desc.shape))
        cntkeypointsT += len(keypoints)
        keypointsTList.append(keypoints)
        descTList.append(desc)
    print("Total count of keypoints: {:d}".format(cntkeypointsT))
    

建立最近邻近似匹配器(FlannBasedMatcher)，准备进行目标图像与原图中的滑窗之间的特征点匹配

In [ ]:
    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    indexParams = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    searchParams = dict(checks = 50)
    matcher = cv.FlannBasedMatcher(indexParams, searchParams)
    

### 建立图像金字塔
可建立图像金字塔（按比例放大、缩小的一系列原图像），用于在原图像中定位不同尺寸的目标。此处略去此步骤。

### 目标的粗略定位
在原图像中按照目标图像的大小放置滑动窗口。滑窗从原图像的左上角开始，向右方、下方不断移动。为减少运算量，此处将垂直、水平移动步长（STEP）均设为5。对于每个滑窗位置，使用SIFT特征检测器检测特征点和计算描述子（同样按图像的BGR三通道分别进行），再使用最近邻近似匹配器进行目标图像与滑窗之间的特征点匹配。为筛选出优良的匹配特征点，此处采用Lowe算法计算每个匹配特征点的最近邻与次近邻之比（LOWE_RATIO）。当比值低于某预设值（此处设为0.5）时，认为该匹配特征点为优良特征点。当目标与滑窗的优良匹配特征点不少于目标的特征点总数的一定比例（此处设为MIN_GOOD_MATCH_RATIO = 0.1）时，将此滑窗设为候选目标区域。

**最近邻近似匹配器（FLANN based Matcher）的优点是匹配速度快；缺点是匹配不够精确。此处也可使用暴力匹配器（Brute-Force Matcher）。暴力匹配器的优点是匹配精度高；缺点是运算量大，匹配速度低。**

In [ ]:
    original1 = original.copy()
    candRegionList = []
    
    for b in range(h_t - 1, h_o, STEP[0]):
        for r in range(w_t - 1, w_o, STEP[1]):
            window = original[b+1-h_t:b+1, r+1-w_t:r+1]
            goodMatches = []
            winFlag = True
            for c in range(3):
                keypoints, desc = sift.detectAndCompute(window[..., c], None)
                colflag = False
                if len(keypoints) >= COUNT_OF_BEST_MATCHES_PER_DESC:
                    matches = matcher.knnMatch(descTList[c], desc, k = COUNT_OF_BEST_MATCHES_PER_DESC)
                    for m, n in matches:
                        if m.distance < LOWE_RATIO * n.distance:
                            goodMatches.append(m)
                            colflag = True
                if not colflag:
                    winFlag = False
                    break
            if not winFlag:
                continue
            if len(goodMatches) >= MIN_GOOD_MATCH_RATIO * cntkeypointsT:
                cv.rectangle(original1, (r - w_t + 1, b - h_t + 1), (r, b), (0, 0, 255))
                candRegionList.append((b - h_t + 1, r - w_t + 1, b, r))
    cv.imshow('Candidate regions - 1', original1)
    print("Count of Candidate regions - 1: {:d}".format(len(candRegionList)))
    

### 丢弃重叠候选区域
对于原图中的同一个目标，往往会产生多个重叠的候选目标区域。此时，需要将这些重叠的候选区域丢弃至只剩一个，以方便后续的处理。对于每个候选区域，计算它与所有其它候选区域的水平距离与垂直距离。若水平距离与垂直距离均小于目标的宽度与高度的一定比例（此处设为DISTANCE_COEFFICIENT = 0.5）时，立即将此候选区域丢弃，则后续的候选区域不再与之计算距离，得以保留。

In [ ]:
    original2 = original.copy()
    candRegionList2 = []
    
    for i in range(len(candRegionList)):
        discardFlag = False
        for j in range(i + 1, len(candRegionList)):
            dist_h = abs(candRegionList[i][1] - candRegionList[j][1])
            dist_v = abs(candRegionList[i][0] - candRegionList[j][0])
            if (dist_h < DISTANCE_COEFFICIENT * w_t) and (dist_v < DISTANCE_COEFFICIENT * h_t):
                discardFlag = True
                break
        if not discardFlag:
            cv.rectangle(original2, (candRegionList[i][1], candRegionList[i][0]), 
                         (candRegionList[i][3], candRegionList[i][2]), (0, 0, 255))
            candRegionList2.append(candRegionList[i])
    cv.imshow('Candidate regions - 2', original2)
    print("Count of Candidate regions - 2: {:d}".format(len(candRegionList2)))
    

### 目标的精确定位
对于剩下的候选目标区域，可认为目标就在其附近。因此，对于每个候选区域，外扩其边界（四至外扩宽度或高度的一定比例，此处设为SEARCH_COEFFICIENT = 0.6，并注意外扩后的边界不要越出原图像的边），在原图像的灰度图像中截取下来，再通过滑窗法与目标图像的灰度图像进行匹配。此处使用灰度图像进行匹配是为了减少运算量。此处使用最简单的计算欧氏距离方法进行模式匹配。所有滑窗中，与目标图像的欧氏距离为最小者，认为是目标区域的精确位置。这样，所有候选区域都能得到其精确位置。

In [ ]:
    original3 = original.copy()
    detectedObjectList = []
    
    for candRegion in candRegionList2:
        croppedGrayTop = int(candRegion[0]-SEARCH_COEFFICIENT*h_t)
        if croppedGrayTop < 0:
            croppedGrayTop = 0
        croppedGrayLeft = int(candRegion[1]-SEARCH_COEFFICIENT*w_t)
        if croppedGrayLeft < 0:
            croppedGrayLeft = 0
        croppedGrayBottom = int(candRegion[2]+SEARCH_COEFFICIENT*h_t)
        if croppedGrayBottom >= h_o:
            croppedGrayBottom = h_o - 1
        croppedGrayRight = int(candRegion[3]+SEARCH_COEFFICIENT*w_t)
        if croppedGrayRight >= w_o:
            croppedGrayRight = w_o - 1
        w_c = croppedGrayRight - croppedGrayLeft + 1
        h_c = croppedGrayBottom - croppedGrayTop + 1
        croppedGray = originalGray[croppedGrayTop:croppedGrayBottom+1, croppedGrayLeft:croppedGrayRight+1]
        minEuclideanDist = np.inf
        for b in range(h_t - 1, h_c):
            for r in range(w_t - 1, w_c):
                window = croppedGray[b+1-h_t:b+1, r+1-w_t:r+1]
                euclideanDist = np.sqrt(((window - targetGray) ** 2).sum())
                if euclideanDist < minEuclideanDist:
                    minEuclideanDist = euclideanDist
                    matchedWindow = (b, r)
        detectedObjectList.append((croppedGrayTop + matchedWindow[0] - h_t + 1, croppedGrayLeft + matchedWindow[1] - w_t + 1, 
                                   croppedGrayTop + matchedWindow[0], croppedGrayLeft + matchedWindow[1]))
    for detectedObject in detectedObjectList:
        cv.rectangle(original3, (detectedObject[1], detectedObject[0]), 
                     (detectedObject[3], detectedObject[2]), (0, 0, 255))
    cv.imshow('Detected objects', original3)
    print("Count of detected objects: {:d}".format(len(detectedObjectList)))
    

返回所有目标区域的位置（左上角、右下角的垂直、水平坐标）。

In [ ]:
    return detectedObjectList

调用函数ObjDet_Tom()，打印输出的结果，并记录执行时间。

In [ ]:
if __name__ is "__main__":
    import sys
    import time
    
    filename1, filename2 = sys.argv[1], sys.argv[2]
    
    # Time the execution of the function
    start = time.time()
    retList = ObjDet_Tom(filename1, filename2)
    end = time.time()
    
    print("-" * 30)
    print("Time elapsed: {:.3f} s".format(end - start))
    print("Count of detected objects: {:d}".format(len(retList)))
    print("Regions of detected objects: ")
    for ret in retList:
        print(ret)

# End of file